dane w formacie .mat do pobrania na http://ufldl.stanford.edu/housenumbers/?fbclid=IwAR1wQkkv6beOR5wqTR4xGa-CRbYDsUi3PWkFk6-ZmzuZnPm6OUNnxMzr4bw

In [1]:
using PyCall
using ImageView
using Flux
using Flux, Flux.Data.FashionMNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra
scipy_io = pyimport("scipy.io")
plt = pyimport("matplotlib.pyplot")
np = pyimport("numpy")
#mat73 = pyimport("mat73")

PyObject <module 'matplotlib.pyplot' from 'C:\\Program Files\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [2]:
test = scipy_io.loadmat("test.mat")
train = scipy_io.loadmat("train.mat")

Dict{Any, Any} with 5 entries:
  "__header__"  => "MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Dec…
  "__globals__" => Any[]
  "X"           => UInt8[0x21 0x0f … 0x52 0x59; 0x1c 0x0e … 0x43 0x53; … ; 0x54…
  "__version__" => "1.0"
  "y"           => UInt8[0x01; 0x09; … ; 0x06; 0x09]

In [3]:
X_train, y_train = train["X"], train["y"]
X_test, y_test = test["X"], test["y"]

(UInt8[0x26 0x27 … 0x2a 0x27; 0x27 0x27 … 0x2b 0x27; … ; 0x32 0x33 … 0x2d 0x2a; 0x30 0x30 … 0x2c 0x2a]

UInt8[0x67 0x68 … 0x67 0x61; 0x68 0x68 … 0x65 0x61; … ; 0x75 0x76 … 0x74 0x71; 0x73 0x73 … 0x73 0x71]

UInt8[0x3c 0x3d … 0x3e 0x39; 0x3d 0x3d … 0x3f 0x39; … ; 0x4c 0x4d … 0x48 0x45; 0x4a 0x4a … 0x47 0x47]

UInt8[0x81 0x7f … 0x7b 0x7b; 0x86 0x85 … 0x7e 0x7d; … ; 0x8a 0x82 … 0x65 0x70; 0x83 0x7a … 0x6f 0x7b]

UInt8[0x8e 0x8f … 0x86 0x87; 0x96 0x95 … 0x89 0x89; … ; 0x98 0x90 … 0x6b 0x76; 0x91 0x88 … 0x76 0x82]

UInt8[0x99 0x98 … 0x9c 0x9d; 0xa0 0x9e … 0x9d 0x9e; … ; 0xa5 0x9d … 0x86 0x90; 0x9e 0x95 … 0x90 0x9c]

UInt8[0x96 0x96 … 0x96 0x93; 0x96 0x97 … 0x99 0x96; … ; 0x93 0x93 … 0x8b 0x8a; 0x9f 0x9f … 0x98 0x98]

UInt8[0xa0 0xa3 … 0xab 0xa9; 0xa0 0xa3 … 0xab 0xa9; … ; 0xa5 0xa4 … 0x9b 0x9b; 0xb1 0xb1 … 0xa8 0xa8]

UInt8[0xa9 0xaa … 0xb5 0xb4; 0xa9 0xa9 … 0xb4 0xb3; … ; 0xb3 0xb3 … 0xaa 0xab; 0xbd 0xbe … 0xb6 0xb7]

...

UInt8[0x73 0x74 … 0x72 0x73; 0x7a 0x7b … 0x76 0x77; … ; 0x75 0x75 …

In [4]:
X_train = permutedims(X_train, (4,1,2,3)) #nowy rozmiar tensora: 73257x32x32x3
X_test = permutedims(X_test, (4,1,2,3))

26032×32×32×3 Array{UInt8, 4}:
[:, :, 1, 1] =
 0x26  0x27  0x26  0x26  0x26  0x27  …  0x33  0x33  0x33  0x34  0x32  0x30
 0x81  0x86  0x8d  0x93  0x98  0x9b     0xae  0xa7  0x9e  0x93  0x8a  0x83
 0x96  0x96  0x90  0x82  0x67  0x5b     0x6f  0x7a  0x7f  0x87  0x93  0x9f
 0x82  0x89  0x94  0x9f  0xa2  0xa0     0x71  0x62  0x59  0x5f  0x76  0x90
 0xb1  0xb1  0xaf  0xaf  0xb0  0xb2     0xb0  0xb0  0xae  0xae  0xad  0xad
 0x4b  0x49  0x45  0x46  0x4a  0x4d  …  0x56  0x57  0x56  0x56  0x56  0x56
 0x08  0x0b  0x0c  0x0d  0x0e  0x0e     0x11  0x0f  0x0d  0x0d  0x0d  0x0e
 0x26  0x26  0x26  0x25  0x24  0x24     0x1c  0x1c  0x1e  0x1e  0x1f  0x20
 0xb4  0xb5  0xb7  0xb8  0xb8  0xb9     0xb5  0xb5  0xb5  0xb5  0xb5  0xb5
 0xb7  0xb6  0xb6  0xb7  0xb7  0xb6     0xb6  0xb0  0xab  0xad  0xb1  0xb3
 0xbc  0xbe  0xb4  0x92  0x6c  0x54  …  0x61  0x67  0x79  0x92  0xa3  0xac
 0x2b  0x2b  0x28  0x21  0x23  0x25     0x19  0x1b  0x1d  0x1e  0x1f  0x20
 0x25  0x23  0x21  0x22  0x24  0x24     0x30  0x30  0x

In [6]:
#X_train2 = X_train ./ 255

73257×32×32×3 Array{Float64, 4}:
[:, :, 1, 1] =
 0.129412   0.109804   0.156863   …  0.337255   0.329412   0.333333
 0.329412   0.333333   0.32549       0.392157   0.403922   0.403922
 0.0745098  0.0823529  0.0823529     0.345098   0.345098   0.329412
 0.0705882  0.0588235  0.133333      0.921569   0.854902   0.556863
 0.423529   0.423529   0.427451      0.458824   0.466667   0.470588
 0.517647   0.505882   0.47451    …  0.443137   0.486275   0.505882
 0.32549    0.337255   0.345098      0.337255   0.341176   0.337255
 0.505882   0.435294   0.423529      0.403922   0.407843   0.403922
 0.164706   0.145098   0.121569      0.219608   0.231373   0.219608
 0.27451    0.317647   0.333333      0.470588   0.419608   0.341176
 0.329412   0.333333   0.337255   …  0.34902    0.34902    0.34902
 0.34902    0.32549    0.298039      0.462745   0.462745   0.462745
 0.0862745  0.0784314  0.0784314     0.0745098  0.0784314  0.0784314
 ⋮                                ⋱             ⋮          
 0.44313

In [13]:
imshow(X_train_gr[4,:,:,:]) #obrazek jest kolorowy

Dict{String, Any} with 4 entries:
  "gui"         => Dict{String, Any}("window"=>GtkWindowLeaf(name="", parent, w…
  "roi"         => Dict{String, Any}("redraw"=>385: "map(clim-mapped image, inp…
  "annotations" => 334: "input-117" = Dict{UInt64, Any}() Dict{UInt64, Any} 
  "clim"        => 333: "CLim" = CLim{Float64}(59.968, 254.701) CLim{Float64} 

In [6]:
#funkcja przerabia obrazki na ostrzejsze - bardziej czarno-białe (znalezione w internetach)
function rgb2gray(images)
    return np.expand_dims(np.dot(images, [0.2990, 0.5870, 0.1140]), axis=3)
end

rgb2gray (generic function with 1 method)